In [130]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [131]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from scipy.stats import loguniform

In [132]:
# Define directories for training and testing data
base_dir = 'archive'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


In [133]:
# Define image dimensions and batch size
img_width, img_height = 32, 32
batch_size = 16

In [134]:
# Create ImageDataGenerator for training and testing data (rescale for normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [135]:
# Generate training data and separate validation set
train_generator, validation_generator = train_test_split(
    train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    ),
    test_size=0.2,  # Adjust validation split as needed
    random_state=42  # Set random state for reproducibility
)

# Generate testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)


Found 24176 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.


In [136]:
# Function for defining the CNN model
def create_model(n_hidden=1, n_neurons=30, learning_rate=3e-3):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
    model.add(MaxPooling2D((2, 2)))
    for _ in range(n_hidden):
        model.add(Conv2D(n_neurons, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [137]:
# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [138]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": list(range(10, 100)),  # Adjusted range for neurons
    "learning_rate": loguniform(1e-5, 1e-2)  # Define learning rate as loguniform distribution
}

In [139]:
# Perform randomized search cross-validation
rnd_search_cv = RandomizedSearchCV(KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0),
                                   param_distribs, n_iter=10, cv=3)

In [140]:
# Fit the model with validation data and early stopping
rnd_search_cv.fit(train_generator, validation_data=validation_generator, epochs=100, callbacks=[early_stopping])

ValueError: Invalid parameter learning_rate for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(learning_rate=0.0018609833419401165)`
Check the list of available parameters with `estimator.get_params().keys()`